<a href="https://colab.research.google.com/github/ashamril/Text-Analytics/blob/master/Malaysia_Covid19_Ringkasan_Kematian_web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Web scapping data (table) Ringkasan Kematian from https://kpkesihatan.com/category/communicable-disease/**

In [1]:
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import requests
import calendar
import re
from collections import Counter
from wordcloud import WordCloud
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict  
from pandas import DataFrame
import plotly.express as px
import numpy as np
from datetime import date, timedelta

**Data collection**

In [2]:
# # For testing
# URL = f"https://kpkesihatan.com/2021/2/9/kenyataan-akhbar-kpk-9-februari-2021-situasi-semasa-jangkitan-penyakit-coronavirus-2019-covid-19-di-malaysia/"

# response = requests.get(URL)
# if response.status_code == 200:
#   html_text = response.text

# soup = BeautifulSoup(html_text)
# tables = soup.findAll('table')

In [3]:
# # For testing
# for table in tables:
#     if "No. Kematian" in table.text:
#       selected_table = table
#       for thead in selected_table:
#         thead.name = "tbody"
#         dfRead = pd.read_html(str(selected_table), header=None)[0]
        
# print(type(dfRead))
# dfRead

In [4]:
def collectLatarBelakang(day, month, mth, year):
  global dfAll
  URL = f"https://kpkesihatan.com/{year}/{month}/{day}/kenyataan-akhbar-kpk-{day}-{mth}-{year}-situasi-semasa-jangkitan-penyakit-coronavirus-2019-covid-19-di-malaysia"

  response = requests.get(URL)
  if response.status_code == 200:
    html_text = response.text
    soup = BeautifulSoup(html_text)
    tables = soup.findAll('table')
  
    for table in tables:
      if "No. Kematian" in table.text:
        selected_table = table
        for thead in selected_table:
          thead.name = "tbody"
          dfRead = pd.read_html(str(selected_table), header=None)[0]
          dfRead['Tarikh Kematian'] = "{} {} {}".format(day, mth, year)
          dfAll = pd.concat([dfAll, dfRead])
          return dfAll
      else:
        #print("Tiada kematian pada {} {} {}".format(day, mth, year))
        tiadaKematian.append("{} {} {}".format(year, month, day))

  else:
    print("Link 404! {}".format(URL))
    

In [5]:
# dfAll = pd.DataFrame()
# tiadaKematian = []

# # # Dec OK
# for i in range(1, 31):
#   day = i
#   month = 12
#   year = 2020
#   mth = "disember"
#   collectLatarBelakang(day, month, str(mth), year)

# # Jan OK
# for i in range(1, 31):
#   day = i
#   month = 1
#   year = 2021
#   mth = "januari"
#   collectLatarBelakang(day, month, str(mth), year)

# # Feb OK
# for i in range(1, 28):
#   day = i
#   month = 2
#   year = 2021
#   mth = "februari"
#   collectLatarBelakang(day, month, str(mth), year)

# dfAll

In [6]:
dfAll = pd.DataFrame()
tiadaKematian = []
months_name = {
  "01": "januari",
  "02": "februari",
  "03": "mac",
  "04": "april",
  "05": "mei",
  "06": "jun",
  "07": "julai",
  "08": "ogos",
  "09": "september",
  "10": "oktober",
  "11": "november",
  "12": "disember"}

# Data collection start from start_date to end_date
start_date = date(2020, 1, 1)
end_date = date(2021, 2, 28)
delta = timedelta(days=1)
while start_date <= end_date:
    day = start_date.strftime("%d")
    if int(day) < 10:
      day = day[1]
    month = start_date.strftime("%m")
    mth = months_name[month]
    year = start_date.strftime("%Y")
    start_date += delta
    collectLatarBelakang(day, month, str(mth), year)

dfAll

Link 404! https://kpkesihatan.com/2020/01/1/kenyataan-akhbar-kpk-1-januari-2020-situasi-semasa-jangkitan-penyakit-coronavirus-2019-covid-19-di-malaysia
Link 404! https://kpkesihatan.com/2020/01/2/kenyataan-akhbar-kpk-2-januari-2020-situasi-semasa-jangkitan-penyakit-coronavirus-2019-covid-19-di-malaysia
Link 404! https://kpkesihatan.com/2020/01/3/kenyataan-akhbar-kpk-3-januari-2020-situasi-semasa-jangkitan-penyakit-coronavirus-2019-covid-19-di-malaysia
Link 404! https://kpkesihatan.com/2020/01/4/kenyataan-akhbar-kpk-4-januari-2020-situasi-semasa-jangkitan-penyakit-coronavirus-2019-covid-19-di-malaysia
Link 404! https://kpkesihatan.com/2020/01/5/kenyataan-akhbar-kpk-5-januari-2020-situasi-semasa-jangkitan-penyakit-coronavirus-2019-covid-19-di-malaysia
Link 404! https://kpkesihatan.com/2020/01/6/kenyataan-akhbar-kpk-6-januari-2020-situasi-semasa-jangkitan-penyakit-coronavirus-2019-covid-19-di-malaysia
Link 404! https://kpkesihatan.com/2020/01/7/kenyataan-akhbar-kpk-7-januari-2020-situasi-

,0,1,2,3,4,5,6,Tarikh Kematian
0,Kes No. Kematian,Kes No.,Negeri,Jantina (Warganegara),Umur (tahun),Hospital,Latar Belakang,26 oktober 2020
1,230,14925,Sabah,Lelaki (WN),67,Hospital Duchess of Kent,Sinusitis,26 oktober 2020
2,231,15854,Sabah,Lelaki (WN),71,Hospital Queen Elizabeth,"Penyakit kencing manis, darah tinggi dan penya...",26 oktober 2020
3,232,16508,Sabah,Perempuan (WN),51,Hospital Queen Elizabeth,"Penyakit kencing manis, darah tinggi dan disli...",26 oktober 2020
4,233,26212,Sabah,Lelaki (WN),61,Hospital Tuaran,Orang Kelainan Upaya (buta),26 oktober 2020
...,...,...,...,...,...,...,...,...
5,1126,280631,Selangor,Lelaki (WN),64,Hospital Kuala Lumpur,"Penyakit darah tinggi, kencing manis, jantung,...",28 februari 2021
6,1127,296200,Selangor,Perempuan (WN),88,Pusat Perubatan Univerisiti Malaya,Osteoporosis,28 februari 2021
7,1128,296663,Selangor,Lelaki (WN),57,Hospital Selayang,"Penyakit darah tinggi, kencing manis, dislipid...",28 februari 2021
8,1129,297934,Perak,Lelaki (WN),69,Hospital Taiping,Penyakit darah tinggi dan kencing manis,28 februari 2021


**No death dates**

In [7]:
x = np.array(tiadaKematian)
uniqDate = np.unique(x)
uniqDate.sort()
print(uniqDate)

['2020 03 1' '2020 03 11' '2020 03 12' '2020 03 13' '2020 03 14'
 '2020 03 2' '2020 03 20' '2020 03 21' '2020 03 22' '2020 03 23'
 '2020 03 24' '2020 03 25' '2020 03 26' '2020 03 27' '2020 03 28'
 '2020 03 29' '2020 03 30' '2020 03 31' '2020 03 4' '2020 03 5'
 '2020 03 6' '2020 03 7' '2020 03 9' '2020 04 1' '2020 04 10' '2020 04 11'
 '2020 04 12' '2020 04 13' '2020 04 14' '2020 04 15' '2020 04 17'
 '2020 04 18' '2020 04 19' '2020 04 2' '2020 04 20' '2020 04 21'
 '2020 04 22' '2020 04 23' '2020 04 24' '2020 04 25' '2020 04 26'
 '2020 04 27' '2020 04 28' '2020 04 29' '2020 04 3' '2020 04 30'
 '2020 04 4' '2020 04 5' '2020 04 6' '2020 04 7' '2020 04 8' '2020 04 9'
 '2020 05 1' '2020 05 10' '2020 05 11' '2020 05 12' '2020 05 14'
 '2020 05 15' '2020 05 16' '2020 05 17' '2020 05 18' '2020 05 19'
 '2020 05 2' '2020 05 20' '2020 05 21' '2020 05 22' '2020 05 23'
 '2020 05 24' '2020 05 25' '2020 05 26' '2020 05 27' '2020 05 29'
 '2020 05 3' '2020 05 30' '2020 05 31' '2020 05 4' '2020 05 5' '2020

**Data preparation/cleaning**

In [8]:
dfAll.columns=["No. Kematian", "No. Kes", "Negeri", "Jantina (Warganegara)", "Umur (tahun)", "Hospital / Tempat Kematian", "Latar Belakang", "Tarikh Kematian"]

In [9]:
dfAll.loc[dfAll["Latar Belakang"].str.contains('buang', flags=re.IGNORECASE)]

,No. Kematian,No. Kes,Negeri,Jantina (Warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang,Tarikh Kematian
11,907,235176,Selangor,Perempuan (WN),67,Hospital Serdang,"Penyakit kencing manis, darah tinggi, jantung,...",9 februari 2021


In [247]:
df_filtered = dfAll[dfAll['No. Kematian'] != 'No. Kematian'] 
df_filtered = dfAll[dfAll['No. Kematian'] != 'Kes No. Kematian'] 

In [248]:
df_filtered = df_filtered.reset_index()
df_filtered = df_filtered.drop('index', axis = 1) 
df_filtered

,No. Kematian,No. Kes,Negeri,Jantina (Warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang,Tarikh Kematian
0,230,14925,Sabah,Lelaki (WN),67,Hospital Duchess of Kent,Sinusitis,26 oktober 2020
1,231,15854,Sabah,Lelaki (WN),71,Hospital Queen Elizabeth,"Penyakit kencing manis, darah tinggi dan penya...",26 oktober 2020
2,232,16508,Sabah,Perempuan (WN),51,Hospital Queen Elizabeth,"Penyakit kencing manis, darah tinggi dan disli...",26 oktober 2020
3,233,26212,Sabah,Lelaki (WN),61,Hospital Tuaran,Orang Kelainan Upaya (buta),26 oktober 2020
4,234,25353,Sabah,Lelaki (BWN),50,Hospital Queen Elizabeth,"Penyakit darah tinggi, buah pinggang kronik da...",26 oktober 2020
...,...,...,...,...,...,...,...,...
1003,1126,280631,Selangor,Lelaki (WN),64,Hospital Kuala Lumpur,"Penyakit darah tinggi, kencing manis, jantung,...",28 februari 2021
1004,1127,296200,Selangor,Perempuan (WN),88,Pusat Perubatan Univerisiti Malaya,Osteoporosis,28 februari 2021
1005,1128,296663,Selangor,Lelaki (WN),57,Hospital Selayang,"Penyakit darah tinggi, kencing manis, dislipid...",28 februari 2021
1006,1129,297934,Perak,Lelaki (WN),69,Hospital Taiping,Penyakit darah tinggi dan kencing manis,28 februari 2021


In [249]:
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace("dan", ",")
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace(" ,", ",")
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace(", ", ",")
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace(",,", ",")
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace("–", "tiada latar belakang")
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace("Penyakit", "")
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace("buang", "buah")
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace("Buang", "buah")
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace("demensia", "dementia")
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace("Demensia", "dementia")
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace("’s", "")
df_filtered['Hospital / Tempat Kematian'] = df_filtered['Hospital / Tempat Kematian'].str.replace("Jenazah di bawa ke ", "")
df_filtered['Hospital / Tempat Kematian'] = df_filtered['Hospital / Tempat Kematian'].str.replace("Jenazah dibawa ke ", "")
df_filtered['Negeri'] = df_filtered['Negeri'].str.replace("W.P.", "WP")

In [250]:
df_filtered['Jantina (Warganegara)'] = df_filtered['Jantina (Warganegara)'].str.replace("Lelak ", "Lelaki")
df_filtered['Jantina (Warganegara)'] = df_filtered['Jantina (Warganegara)'].str.replace("Lelaki ", "Lelaki")
df_filtered['Jantina (Warganegara)'] = df_filtered['Jantina (Warganegara)'].str.replace("Perempuan ", "Perempuan")
# Remove white space
df_filtered['Jantina (Warganegara)'] = df_filtered['Jantina (Warganegara)'].str.replace("\s+", "")

In [251]:
df_filtered.loc[df_filtered["Latar Belakang"].str.contains('buang', flags=re.IGNORECASE)]

,No. Kematian,No. Kes,Negeri,Jantina (Warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang,Tarikh Kematian


In [252]:
df_filtered

,No. Kematian,No. Kes,Negeri,Jantina (Warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang,Tarikh Kematian
0,230,14925,Sabah,Lelaki(WN),67,Hospital Duchess of Kent,Sinusitis,26 oktober 2020
1,231,15854,Sabah,Lelaki(WN),71,Hospital Queen Elizabeth,"kencing manis,darah tinggi,penyakit jantung",26 oktober 2020
2,232,16508,Sabah,Perempuan(WN),51,Hospital Queen Elizabeth,"kencing manis,darah tinggi,dislipidemia",26 oktober 2020
3,233,26212,Sabah,Lelaki(WN),61,Hospital Tuaran,Orang Kelainan Upaya (buta),26 oktober 2020
4,234,25353,Sabah,Lelaki(BWN),50,Hospital Queen Elizabeth,"darah tinggi,buah pinggang kronik,penyakit ja...",26 oktober 2020
...,...,...,...,...,...,...,...,...
1003,1126,280631,Selangor,Lelaki(WN),64,Hospital Kuala Lumpur,"darah tinggi,kencing manis,jantung,buah pingg...",28 februari 2021
1004,1127,296200,Selangor,Perempuan(WN),88,Pusat Perubatan Univerisiti Malaya,Osteoporosis,28 februari 2021
1005,1128,296663,Selangor,Lelaki(WN),57,Hospital Selayang,"darah tinggi,kencing manis,dislipidemia,obesi...",28 februari 2021
1006,1129,297934,Perak,Lelaki(WN),69,Hospital Taiping,"darah tinggi,kencing manis",28 februari 2021


**Group by**

In [253]:
grpJantina = df_filtered.groupby(['Jantina (Warganegara)'])
grpJantina.get_group('Lelaki(WN)') 

,No. Kematian,No. Kes,Negeri,Jantina (Warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang,Tarikh Kematian
0,230,14925,Sabah,Lelaki(WN),67,Hospital Duchess of Kent,Sinusitis,26 oktober 2020
1,231,15854,Sabah,Lelaki(WN),71,Hospital Queen Elizabeth,"kencing manis,darah tinggi,penyakit jantung",26 oktober 2020
3,233,26212,Sabah,Lelaki(WN),61,Hospital Tuaran,Orang Kelainan Upaya (buta),26 oktober 2020
7,237,27478,Sabah,Lelaki(WN),82,Hospital Tawau,"paru-paru obstruktif kronik (COPD),buah pingg...",27 oktober 2020
9,239,12156,Sabah,Lelaki(WN),66,Hospital Queen Elizabeth,"kencing manis,darah tinggi,dislipidemia",28 oktober 2020
...,...,...,...,...,...,...,...,...
1000,1123,209226,Sarawak,Lelaki(WN),81,Hospital Sibu,"darah tinggi,kencing manis",28 februari 2021
1002,1125,272051,Kelantan,Lelaki(WN),68,"Hospital Sultan Ismail Petra, Kuala Krai",kencing manis,28 februari 2021
1003,1126,280631,Selangor,Lelaki(WN),64,Hospital Kuala Lumpur,"darah tinggi,kencing manis,jantung,buah pingg...",28 februari 2021
1005,1128,296663,Selangor,Lelaki(WN),57,Hospital Selayang,"darah tinggi,kencing manis,dislipidemia,obesi...",28 februari 2021


In [254]:
grpNegeri = df_filtered.groupby(['Negeri'])
# Displat first value
grpNegeri.first()

,No. Kematian,No. Kes,Jantina (Warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang,Tarikh Kematian
Negeri,,,,,,,
Johor,323,43767,Perempuan(WN),62,"Hospital Enche’ Besar Hajjah Khalsom, Kluang","kencing manis,darah tinggi,penyakit buah ping...",19 november 2020
Kedah,279,35303,Lelaki(WN),54,Hospital Sultanah Bahiyah,"darah tinggi,buah pinggang kronik,strok",6 november 2020
Kelantan,372,56656,Perempuan(WN),72,"Hospital Raja Perempuan Zainab II, Kota Bharu","darah tinggi,dementia",3 disember 2020
Melaka,451,101553,Lelaki(WN),70,Hospital Melaka,"kencing manis,darah tinggi,Benign Prostatic H...",26 disember 2020
Negeri,No. Kematian,Kes No.,Jantina(Warganegara),Umur (tahun),Hospital,Latar Belakang,30 oktober 2020
Negeri Sembilan,356,60507,Lelaki(WN),89,"Disahkan kematian di Hospital Tuanku Ja’afar, ...",Kanser perut,29 november 2020
Pahang,498,108319,Lelaki(WN),45,"Hospital Tengku Ampuan Afzan, Kuantan","hati,Retroviral",4 januari 2021
Perak,266,16846,Lelaki(WN),79,Hospital Raja Permaisuri Bainun,"darah tinggi,dislipidemia,penyakit buah pinggang",4 november 2020
Pulau Pinang,338,49641,Lelaki(WN),84,Hospital Pulau Pinang,"darah tinggi,penyakit kronik paru-paru",24 november 2020


In [255]:
grpNegeri.get_group('Selangor') 

,No. Kematian,No. Kes,Jantina (Warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang,Tarikh Kematian
16,246,28252,Perempuan(WN),61,Hospital Sungai Buloh,"kencing manis,darah tinggi",28 oktober 2020
43,272,35267,Lelaki(WN),61,Hospital Selayang,"kencing manis,darah tinggi,asma,penyakit jantung",5 november 2020
162,366,23536,Perempuan(WN),72,Hospital Sungai Buloh,"darah tinggi,dislipidemia,obesiti",3 disember 2020
169,373,62601,Perempuan(WN),77,Hospital Sungai Buloh,"darah tinggi,asma,dislipidemia,strok",3 disember 2020
170,374,63565,Lelaki(WN),81,Hospital Sungai Buloh,"darah tinggi,kencing manis,strok,penyakit kem...",3 disember 2020
...,...,...,...,...,...,...,...
991,1115,288447,Perempuan(WN),86,Hospital Sungai Buloh,"kencing manis,darah tinggi",27 februari 2021
992,1116,288461,Lelaki(WN),63,Hospital Sungai Buloh,darah tinggi,27 februari 2021
1003,1126,280631,Lelaki(WN),64,Hospital Kuala Lumpur,"darah tinggi,kencing manis,jantung,buah pingg...",28 februari 2021
1004,1127,296200,Perempuan(WN),88,Pusat Perubatan Univerisiti Malaya,Osteoporosis,28 februari 2021


In [278]:
nByAge = df_filtered.groupby(["Negeri"], sort=False)["No. Kematian"].count() 
nByAge

Negeri
Sabah                         272
Selangor                      259
Negeri                        110
Perak                          34
Kedah                          14
Wilayah Persekutuan Labuan      1
Johor                          75
Pulau Pinang                   14
WP Labuan                       9
Negeri Sembilan                22
WP Kuala Lumpur                80
WP Putrajaya                    7
Kelantan                       16
Melaka                         14
Pahang                          7
Terengganu                     10
Sarawak                        64
Name: No. Kematian, dtype: int64

In [277]:
nByDate = df_filtered.groupby(["Tarikh Kematian"], sort=False)["No. Kematian"].count()
nByDate

Tarikh Kematian
26 oktober 2020      7
27 oktober 2020      2
28 oktober 2020      8
30 oktober 2020      4
2 november 2020      2
                    ..
24 februari 2021    13
25 februari 2021    13
26 februari 2021    12
27 februari 2021    11
28 februari 2021    10
Name: No. Kematian, Length: 120, dtype: int64

**Save to csv file**

In [258]:
dfRingkasanKematian = df_filtered.copy()
dfRingkasanKematian.to_csv("dfRingkasanKematian.csv", sep=',', mode='a', index=False)

In [259]:
# #For testing
# del dfRingkasanKematian
# dfRingkasanKematian = pd.DataFrame()
# for i in range(7, 11):
#   day = i
#   month = 12
#   year = 2020
#   mth = "disember"
#   df = collectLatarBelakang(day, month, str(mth), year)
#   print(df)
#   dfRingkasanKematian = pd.DataFrame.append(dfRingkasanKematian, df)

# dfRingkasanKematian

**Extend latar belakang/penyakit to new columns with True/False**

In [260]:
dfUniqueLatarBelakang = df_filtered.copy()

**Column to list (Unique penyakit)**

In [261]:
lstLatarBelakang = dfUniqueLatarBelakang["Latar Belakang"].to_numpy()

stopwords = ["Penyakit", " Penyakit", "Penyakit ", "penyakit", " penyakit", "penyakit ", "Penyakti", " Penyakti", "Penyakti ", "penyakti", " penyakti", "penyakti "]
lstLatarBelakang = [(lambda x: re.sub(r'|'.join(stopwords), '', x))(x) for x in lstLatarBelakang]
lstLatarBelakang = [element for item in lstLatarBelakang for element in item.split(',')]
lstLatarBelakang = map(str.strip, lstLatarBelakang)
lstLatarBelakang = list(filter(None, lstLatarBelakang))
lstLatarBelakang = [w.replace('.', '') for w in lstLatarBelakang]
lstLatarBelakang = [w.replace("’s", "") for w in lstLatarBelakang]
lstLatarBelakang = [w.replace(u'\xa0', u' ') for w in lstLatarBelakang]
lstLatarBelakang = [w.replace("demensia" ,"dementia") for w in lstLatarBelakang]
lstLatarBelakang = [w.replace("tiggi" ,"tinggi") for w in lstLatarBelakang]
lstUniqueLatarBelakang = np.unique(lstLatarBelakang)
lstUniqueLatarBelakang = [element.lower() for element in lstUniqueLatarBelakang]

In [262]:
lstUniqueLatarBelakang[:20]

['addison disease',
 'adrenal insufficiency',
 'alzheimer',
 'antiphospholipid syndrome',
 'asma',
 'barah otak',
 'bengkak kelenjar prostat (bph)',
 'benign prostatic hyperplasia',
 'benign prostatic hypertrophy',
 'beta thalassemia',
 'chronic obstructive airway disease',
 'critical limb ischemia',
 'cushing syndrome',
 'deep vein thrombosis (dvt)',
 'dementia',
 'dislipidemia',
 'empty sella syndrome',
 'epilepsy',
 'gastroesophageal reflux disease (gerd)',
 'gout']

**List to new columns**

In [263]:
for penyakit in lstUniqueLatarBelakang:
  dfUniqueLatarBelakang[penyakit] = np.where(dfUniqueLatarBelakang["Latar Belakang"].str.contains(penyakit, flags=re.IGNORECASE), True, False)

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning:

This pattern has match groups. To actually get the groups, use str.extract.



In [264]:
dfUniqueLatarBelakang

,No. Kematian,No. Kes,Negeri,Jantina (Warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang,Tarikh Kematian,addison disease,adrenal insufficiency,alzheimer,antiphospholipid syndrome,asma,barah otak,bengkak kelenjar prostat (bph),benign prostatic hyperplasia,benign prostatic hypertrophy,beta thalassemia,chronic obstructive airway disease,critical limb ischemia,cushing syndrome,deep vein thrombosis (dvt),dementia,dislipidemia,empty sella syndrome,epilepsy,gastroesophageal reflux disease (gerd),gout,henoch schonlein purpura,hepatitis b,hipotiroidisme,kanser pangkal rahim,kanser payu dara,kanser perut,kanser usus,laryngeal cancer,latar belakang,limfoma,lymphoma,mediastinal tumour,...,kebutaan,kegagalan jantung,kelenjar prostat,kelenjar tiroid,kemurungan,kencing manis,kronik paru-paru,leukemia,liver cirrhosis,lumpuh,lumpuh anggota,lung fibrosis,masalah pendengaran,mata,mata kronik,medulloblastoma,neurogenic bladder,obstructive sleep apnea,obstruktif paru-paru kronik (coad),obstruktif paru-paru kronik (copd),ostearthritis,osteoatritis,paru-paru kronik obstruktif (coad),paru-paru obstruktif kronik,paru-paru obstruktif kronik (coad),paru-paru obstruktif kronik (copd),polycythaemia,primary aldosteronism,psoriasis,pulmonary embolism,rendah imuniti,sakit jantung,saraf,spondylolisthesis,terlantar,tiada latar belakang,traumatic brain injury,tuberculosis,ulser peptik,ulser perut
0,230,14925,Sabah,Lelaki(WN),67,Hospital Duchess of Kent,Sinusitis,26 oktober 2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,231,15854,Sabah,Lelaki(WN),71,Hospital Queen Elizabeth,"kencing manis,darah tinggi,penyakit jantung",26 oktober 2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,232,16508,Sabah,Perempuan(WN),51,Hospital Queen Elizabeth,"kencing manis,darah tinggi,dislipidemia",26 oktober 2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,233,26212,Sabah,Lelaki(WN),61,Hospital Tuaran,Orang Kelainan Upaya (buta),26 oktober 2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,234,25353,Sabah,Lelaki(BWN),50,Hospital Queen Elizabeth,"darah tinggi,buah pinggang kronik,penyakit ja...",26 oktober 2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

In [265]:
dfUniqueLatarBelakang.loc[dfUniqueLatarBelakang['dementia'] == True] 

,No. Kematian,No. Kes,Negeri,Jantina (Warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang,Tarikh Kematian,addison disease,adrenal insufficiency,alzheimer,antiphospholipid syndrome,asma,barah otak,bengkak kelenjar prostat (bph),benign prostatic hyperplasia,benign prostatic hypertrophy,beta thalassemia,chronic obstructive airway disease,critical limb ischemia,cushing syndrome,deep vein thrombosis (dvt),dementia,dislipidemia,empty sella syndrome,epilepsy,gastroesophageal reflux disease (gerd),gout,henoch schonlein purpura,hepatitis b,hipotiroidisme,kanser pangkal rahim,kanser payu dara,kanser perut,kanser usus,laryngeal cancer,latar belakang,limfoma,lymphoma,mediastinal tumour,...,kebutaan,kegagalan jantung,kelenjar prostat,kelenjar tiroid,kemurungan,kencing manis,kronik paru-paru,leukemia,liver cirrhosis,lumpuh,lumpuh anggota,lung fibrosis,masalah pendengaran,mata,mata kronik,medulloblastoma,neurogenic bladder,obstructive sleep apnea,obstruktif paru-paru kronik (coad),obstruktif paru-paru kronik (copd),ostearthritis,osteoatritis,paru-paru kronik obstruktif (coad),paru-paru obstruktif kronik,paru-paru obstruktif kronik (coad),paru-paru obstruktif kronik (copd),polycythaemia,primary aldosteronism,psoriasis,pulmonary embolism,rendah imuniti,sakit jantung,saraf,spondylolisthesis,terlantar,tiada latar belakang,traumatic brain injury,tuberculosis,ulser peptik,ulser perut
168,372,56656,Kelantan,Perempuan(WN),72,"Hospital Raja Perempuan Zainab II, Kota Bharu","darah tinggi,dementia",3 disember 2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
292,473,98014,Selangor,Lelaki(WN),94,Hospital Sungai Buloh,"kencing manis,darah tinggi,dementia,bronchiec...",1 januari 2021,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
298,478,112584,Perak,Lelaki(WN),82,Hospital Raja Permaisuri Bainun Ipoh,"Chronic Obstructive Airway Disease,dementia,m...",2 januari 2021,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
410,578,128750,Sabah,Perempuan(BWN),88,Hospital Lahad Datu,dementia,14 januari 2021,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
487,647,157810,Pulau Pinang,Perempuan(WN),59,Hospital Kepala Batas,"darah tinggi,dementia",22 januari 2021,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,

In [266]:
dfUniqueLatarBelakang.drop(['Latar Belakang', 'Hospital / Tempat Kematian'], axis=1)

,No. Kematian,No. Kes,Negeri,Jantina (Warganegara),Umur (tahun),Tarikh Kematian,addison disease,adrenal insufficiency,alzheimer,antiphospholipid syndrome,asma,barah otak,bengkak kelenjar prostat (bph),benign prostatic hyperplasia,benign prostatic hypertrophy,beta thalassemia,chronic obstructive airway disease,critical limb ischemia,cushing syndrome,deep vein thrombosis (dvt),dementia,dislipidemia,empty sella syndrome,epilepsy,gastroesophageal reflux disease (gerd),gout,henoch schonlein purpura,hepatitis b,hipotiroidisme,kanser pangkal rahim,kanser payu dara,kanser perut,kanser usus,laryngeal cancer,latar belakang,limfoma,lymphoma,mediastinal tumour,meningioma,mixed connective tissue disease (mctd),...,kebutaan,kegagalan jantung,kelenjar prostat,kelenjar tiroid,kemurungan,kencing manis,kronik paru-paru,leukemia,liver cirrhosis,lumpuh,lumpuh anggota,lung fibrosis,masalah pendengaran,mata,mata kronik,medulloblastoma,neurogenic bladder,obstructive sleep apnea,obstruktif paru-paru kronik (coad),obstruktif paru-paru kronik (copd),ostearthritis,osteoatritis,paru-paru kronik obstruktif (coad),paru-paru obstruktif kronik,paru-paru obstruktif kronik (coad),paru-paru obstruktif kronik (copd),polycythaemia,primary aldosteronism,psoriasis,pulmonary embolism,rendah imuniti,sakit jantung,saraf,spondylolisthesis,terlantar,tiada latar belakang,traumatic brain injury,tuberculosis,ulser peptik,ulser perut
0,230,14925,Sabah,Lelaki(WN),67,26 oktober 2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,231,15854,Sabah,Lelaki(WN),71,26 oktober 2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,232,16508,Sabah,Perempuan(WN),51,26 oktober 2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,233,26212,Sabah,Lelaki(WN),61,26 oktober 2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,234,25353,Sabah,Lelaki(BWN),50,26 oktober 2020,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

**Save to csv file**

In [267]:
dfUniqueLatarBelakang.to_csv("dfUniqueLatarBelakang.csv", sep=',', mode='a', index=False)